# Justin Horan CSE 534 Project - Adaptive Video Streaming Policy

In [3]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

conf = fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Token File,/home/fabric/.tokens.json
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jhoran4_0000159239
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


### Define FABRIC nodes and establish slice

In [4]:
slice_name = "new-video-streaming-policy-" + fablib.get_bastion_username()

clt = "justin" # Client
srv = "vid_db" # Server
rtr = "router" # Router

''' 
Establish 3 distinct nodes for the slice: 
    justin -- Will be the user of the DASH client with the various video streaming policies
    vid_db -- The server that will hold the video to send to romeo
    router -- Will establish the route from juliet to romeo and vice-versa
'''

node_conf = [
    {'name': clt, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']},
    {'name': srv, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']},
    {'name': rtr, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}
]

'''
Establish the network configuration between all 3 nodes
    Subnets: 
        client - 10.10.1.0/24
        server - 10.10.2.0/24
    Specific addresses:
        client - 10.10.1.5
        router - 10.10.1.50
        server - 10.10.2.5
        router - 10.10.2.50
'''

clt_subnet   = "10.10.1.0/24"
srv_subnet   = "10.10.2.0/24"

clt_addr     = "10.10.1.5"
srv_addr     = "10.10.2.5"
rtr_addr_clt = "10.10.1.50"
rtr_addr_srv = "10.10.2.50"

net_conf = [
 {"name": "net" + clt, "subnet": clt_subnet, "nodes": [{"name": clt,  "addr": clt_addr}, {"name": rtr, "addr": rtr_addr_clt}]},
 {"name": "net" + srv, "subnet": srv_subnet, "nodes": [{"name": srv,  "addr": srv_addr}, {"name": rtr, "addr": rtr_addr_srv}]},
]

'''
Establish the routes between the client and the server
    Client to Server
        Any address in 10.10.1.0/24 subnet gets to server through 10.10.2.50 gateway
    Server to Client
        Any address in 10.10.1.0/24 subnet gets to server through 10.10.1.50 gateway
'''

route_conf = [
 {"addr": clt_subnet, "gw": rtr_addr_srv, "nodes": [srv]}, 
 {"addr": srv_subnet, "gw": rtr_addr_clt, "nodes": [clt]}
]

'''
Establish experiment configuration
'''

exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

'''
    Ensure the slice does not exist yet
'''

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)


You already have a slice by this name!
If you previously reserved resources, skip to the 'log in to resources' section.


In [ ]:
'''
Obtain a suitable site for the slice to be hosted on
'''

while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

### Configure the slice properly and submit the request

In [ ]:
'''
Add the nodes to the slice
'''
for n in node_conf:
    slice.add_node(name=n['name'],
                   site = site_name,
                   cores=n['cores'],
                   ram=n['ram'],
                   disk=n['disk'],
                   image=n['image'],)

'''
Configure and add the network interface cards (Basic) for each network
Add each configured network to the slice
'''
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)
    
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure the created slice

In [ ]:
slice = fablib.get_slice(name=slice_name)

# Install all of the packages listed for each node in the configuration
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

# Assign ip addresses to interfaces
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# Store host name and IP addresses of each node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

### Extend slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Draw network topology

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Login to resources

In [21]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

,SSH command
Name,
justin,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fe9c:a2c7
vid_db,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fe7a:209e
router,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fead:f82b


### Execute Experiments via Sending Commands

In [5]:
import pprint

HOME = "/home/fabric/work/CSE534_Project/results/"

RB = "rb/"
BB = "bb/"
SARA = "sara/"
RA = "ra/"

CBR = "cbr/"
INTERRUPTIONS = "interruptions/"
TRACE_M15 = "trace/bus_m15/"
TRACE_B62 = "trace/bus_b62_2/"
TRACE_NYU = "trace/NYU_Campus_Bus/"

slice = fablib.get_slice(name=slice_name)

nodes = slice.get_nodes()
client = nodes[0]
server = nodes[1]
router = nodes[2]

#### Reset Client

In [6]:
stdout, stderr = client.execute("sudo bash reset.sh",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True)

#### Run CBR Experiment

In [ ]:
# Set the Router's Rate to 1000KBit
stdout, stderr = router.execute("bash rate-set.sh 1000Kbit")

##### Rate-Based

In [ ]:
dir_to_save = HOME + RB + CBR

stdout, stderr = client.execute("python3 ~/AStream/dist/client/dash_client.py -m http://vid_db/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True,
                                timeout=180)

client.execute("sudo bash extract_results.sh",
                private_key_file="/home/fabric/work/fabric_config/slice_key",
                quiet=True)

client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")

DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

stdout, stderr = client.execute("sudo bash reset.sh",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True)

##### Buffer-Based

In [ ]:
dir_to_save = HOME + BB + CBR

stdout, stderr = client.execute("python3 ~/AStream/dist/client/dash_client.py -m http://vid_db/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'netflix' -d",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True,
                                timeout=180)

client.execute("sudo bash extract_results.sh",
                private_key_file="/home/fabric/work/fabric_config/slice_key",
                quiet=True)

client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")

DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

stdout, stderr = client.execute("sudo bash reset.sh",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True)

##### SARA

In [ ]:
dir_to_save = HOME + SARA + CBR

stdout, stderr = client.execute("python3 ~/AStream/dist/client/dash_client.py -m http://vid_db/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'sara' -d",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True,
                                timeout=180)

client.execute("sudo bash extract_results.sh",
                private_key_file="/home/fabric/work/fabric_config/slice_key",
                quiet=True)

client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")

DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

stdout, stderr = client.execute("sudo bash reset.sh",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True)

##### Randomized Adaptation

In [ ]:
dir_to_save = HOME + RA + CBR

stdout, stderr = client.execute("python3 ~/AStream/dist/client/dash_client.py -m http://vid_db/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'randomized' -d",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True,
                                timeout=180)

client.execute("sudo bash extract_results.sh",
                private_key_file="/home/fabric/work/fabric_config/slice_key",
                quiet=True)

client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")

DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

stdout, stderr = client.execute("sudo bash reset.sh",
                                private_key_file="/home/fabric/work/fabric_config/slice_key",
                                quiet=True)

#### Run Interruptions Experiment on Terminal

In [43]:
# Download Log files
dir_to_save = HOME + RB + INTERRUPTIONS
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [44]:
# Download Log files
dir_to_save = HOME + BB + INTERRUPTIONS
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [45]:
# Download Log files
dir_to_save = HOME + SARA + INTERRUPTIONS
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [47]:
# Download Log files
dir_to_save = HOME + RA + INTERRUPTIONS
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

#### Run Trace Experiments on Terminal

In [48]:
# Download Log files
dir_to_save = HOME + RB + TRACE_M15
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [51]:
# Download Log files
dir_to_save = HOME + RB + TRACE_B62
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [52]:
# Download Log files
dir_to_save = HOME + RB + TRACE_NYU
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [53]:
# Download Log files
dir_to_save = HOME + BB + TRACE_M15
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [54]:
# Download Log files
dir_to_save = HOME + BB + TRACE_B62
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [55]:
# Download Log files
dir_to_save = HOME + BB + TRACE_NYU
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [56]:
# Download Log files
dir_to_save = HOME + SARA + TRACE_M15
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [58]:
# Download Log files
dir_to_save = HOME + SARA + TRACE_B62
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [59]:
# Download Log files
dir_to_save = HOME + SARA + TRACE_NYU
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [60]:
# Download Log files
dir_to_save = HOME + RA + TRACE_M15
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [62]:
# Download Log files
dir_to_save = HOME + RA + TRACE_B62
client.download_file(dir_to_save + "BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

In [9]:
# Download Log files
dir_to_save = HOME + RA + TRACE_NYU

DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
client.download_file(dir_to_save + "DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

### Import files into client node and/or server node

In [11]:
slice.get_node(clt).upload_file("/home/fabric/work/CSE534_Project/dash_client.py", "/home/ubuntu/AStream/dist/client/dash_client.py")

<SFTPAttributes: [ size=29997 uid=1000 gid=1000 mode=0o100775 atime=1714540973 mtime=1714544047 ]>

In [ ]:
slice.get_node(clt).upload_file("/home/fabric/work/CSE534_Project/netflix_dash.py", "/home/ubuntu/AStream/dist/client/adaptation/netflix_dash.py")

In [ ]:
slice.get_node(clt).upload_file("/home/fabric/work/CSE534_Project/config_dash.py", "/home/ubuntu/AStream/dist/client/config_dash.py")

In [10]:
slice.get_node(clt).upload_file("/home/fabric/work/CSE534_Project/randomized_dash.py", "/home/ubuntu/AStream/dist/client/adaptation/randomized_dash.py")

<SFTPAttributes: [ size=8431 uid=1000 gid=1000 mode=0o100664 atime=1714540973 mtime=1714544044 ]>

In [ ]:
slice.get_node(srv).upload_file("/home/fabric/work/adaptive-video/DASH_BUFFER_LOG-last.csv", "/home/ubuntu/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv")

### Download files from client node and/or server node

In [ ]:
slice.get_node("clt").download_file("/home/fabric/work/adaptive-video/DASH_BUFFER_LOG-last.csv", "/home/ubuntu/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv")

In [ ]:
slice.get_node("srv").download_file("/home/fabric/work/adaptive-video/DASH_BUFFER_LOG-last.csv", "/home/ubuntu/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv")

In [26]:
client.download_file("/home/fabric/work/CSE534_Project/reset.sh", "/home/ubuntu/reset.sh")

In [27]:
client.download_file("/home/fabric/work/CSE534_Project/extract_results.sh", "/home/ubuntu/extract_results.sh")

### Delete Slice

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)